# Trajectory generation using LQR


Suppose that we wish to find a smooth trajectory, in the sense that the derivatives 
exist up to a given order $n$, interpolating a set of $N$ points specified in 
the 3D space $p_i = \left[x_i \quad y_i \quad z_i\right]^{\top}$ and in time 
$T_i$, where $i \in \{1,\ldots,N\}$ and $\{T_i|i \in \{1,\dots,N\}\}$ is an 
increasing sequence. In this live script, we formulate a Linear Quadratic Regulator 
(LQR) problem to provide a solution to such a problem.

We start by defining a piecewise linear trajectory interpolating the points

$$r(t)=p_{i}+\frac{\left(t-T_{i}\right)}{\left(T_{i+1}-T_{i}\right)}\left(p_{i+1}-p_{i}\right), 
\quad \text { for } t \in\left[T_{i}, T_{i+1}\right) \text { and } i \in\{1, 
\ldots, N-1\}$$

and a sampled version of this trajectory at times $k \tau_{1}$, where $\tau_{1}$ is a given sampling period,

$$r_{k}=r\left(k \tau_{1}\right), \text { for } k \in\{0, \ldots, h\} ,$$

where $h=\left\lfloor\frac{T_{N}}{\tau_{1}}\right\rfloor$is the floor of $\frac{T_{N}}{\tau_{1}}$. 
Define the nth derivative of the trajectory $p(t)=\left[\begin{array}{lll}{x(t)} 
& {y(t)} & {z(t)}\end{array}\right]^{\top}$ to be $u(t)=\left[\begin{array}{lll}{u_{x}(t)} 
& {u_{y}(t)} & {u_{z}(t)}\end{array}\right]^{\top}$ i.e, $p^{(n)}(t)=u(t)$ and 
suppose that it is a piecewise constant function

$$u(t)=u_{k}, \quad t \in\left[k \tau_{1},(k+1) \tau_{1}\right), \quad k \in\{0, 
\ldots, h-1\}.$$

The initial position and derivatives of the trajectory up to $n-1$ are assumed 
to be given as $p^{(j)}(0)$, $j \in\{0,1, \ldots, n-1\}$. Let $p_{k}:=p\left(k 
\tau_{1}\right)$, for $k \in\{0, \ldots, h\}$, and consider the following LQR 
problem

$$\min _{\left\{u_{k} | k \in\{0, \ldots, h-1\}\right\}} \sum_{k=0}^{h-1}\left\|p_{k}-r_{k}\right\|^{2}+\rho\left\|u_{k}\right\|^{2}+\left\|p_{h}-r_{h}\right\|^{2}$$

where $\rho$ is a given positive constant.

In order to solve this problem we notice that we can decompose it into three 
problems for x, y, and z. For each we can consider the model

$$ \bar p^{(n)}(t) = \bar{u}(t)$$

where now $\bar{p}$ and $\bar{u}$ are scalars or equivalently

$$ \dot{\bar{x}} = A_c \bar{x}+ B_c\bar{u}$$

where $\bar{x}=[\bar{p} \ \ \dot{\bar{p}}\ \ \dots \ \ \bar{p}^{(n)}]^T$ and

$$ A_c= {\left[ \matrix{ 0 & 1 & 0 & \dots & 0 \cr0 & 0 & 1 & \dots & 0 \cr\dots 
& \dots & \dots & \dots & \dots \cr0 & 0 & 0 & \dots & 0 } \right]},\ \ \ B_c= 
{\left[ \matrix{ 0  \cr 0 \cr \vdots \cr 1 } \right]}.$$

In fact, consider $x_k:=\bar{x}(k\tau_1)$ and

$$ \bar{u}(t) = u_k, \ \ \ \ \ t \in [k\tau_1,(k+1)\tau_1), \ \ k \in \{0,\dots,h-1\}.$$

We can write

$$ x_{k+1} = Ax_k+Bu_k,$$

where $A = e^{A_c\tau_1}$, $B=\int_0^{\tau_1}e^{A_c s}ds$ (these matrices 
can be obtained with c2d.m). Moreover,

$$ p_k = Mx_k, \ \ \ M=\left[\matrix{	1 & 0 & \dots & 0}\right].$$

We are then interested in solving a reference tracking LQR problem 

$$ (\sum_{k=0}^{h-1}\|Mx_k-r_k\|^2 + u_k^T Ru_k +\|Mx_h-r_h\|^2)$$

subject to 

$$ x_{k+1} = Ax_k+Bu_k, \ \ k \in \mathbb{N}_0$$

for a given initial condition. This is a reference tracking problem, see live 
script "Reference tracking LQR''.

The Matlab function `lqrtrajgeneration` below solves the desired problem and 
provides the desired trajectory. 

The input arguments to the function are:

* $P$, a $3 \times N$ vector of points in 3D space such that $P=\left[\begin{array}{llll}{p_{1}} 
& {p_{2}} & {\dots} & {p_{N}}\end{array}\right]$
* $T$, a vector with times $T=\left[\begin{array}{llll}{T_{1}} & {T_{2}} & 
{\dots} & {T_{N}}\end{array}\right]$
* $X0$, a $3 \times n$ matrix with initial trajectory position and derivatives

$$X 0=\left[\begin{array}{llll}{x(0)} & {x^{\prime}(0)} & {\dots} & {x^{(n-1)}(0)} 
\\ {y(0)} & {y^{\prime}(0)} & {\dots} & {y^{(n-1)}(0)} \\ {z(0)} & {z^{\prime}(0)} 
& {\dots} & {z^{(n-1)}(0)}\end{array}\right]$$

* rho = $\rho$
* $\tau_1$ is the sampling period in the problem formulation determining the 
shape of the nth derivative of $p$.
* $N_{12}$ is such that $\tau_{2}=\frac{\tau_{1}}{N_{12}}$ is the output sampling 
period.

The output arguments are:

* xix , a matrix with entries

$$\left[\begin{array}{cccc}{x(0)} & {x\left(\tau_{2}\right)} & {\dots} & {x\left(H 
\tau_{2}\right)} \\ {x^{\prime}(0)} & {x^{\prime}\left(\tau_{2}\right)} & {\dots} 
& {x^{\prime}\left(H \tau_{2}\right)} \\ {x^{(n)}(0)} & {x^{(n)}\left(\tau_{2}\right)} 
& {\dots} & {x^{(n)}\left(H \tau_{2}\right)}\end{array}\right]$$

where $H=N_{12} h$. 

Note that $x^{(n)}\left(H \tau_{2}\right)$ is undefined. Hence, make $x^{(n)}\left(H 
\tau_{2}\right)=x^{(n)}\left((H-1) \tau_{2}\right)$.

* xiy is defined similarly as xix but for the y components of p as

$$\left[\begin{array}{cccc}{y(0)} & {y\left(\tau_{2}\right)} & {\dots} & {y\left(H 
\tau_{2}\right)} \\ {y^{\prime}(0)} & {y^{\prime}\left(\tau_{2}\right)} & {\dots} 
& {y^{\prime}\left(H \tau_{2}\right)} \\ {y^{(n)}(0)} & {y^{(n)}\left(\tau_{2}\right)} 
& {\dots} & {y^{(n)}\left(H \tau_{2}\right)}\end{array}\right]$$

* xiz is also defined similar to xix but for the z component of p as

$$\left[\begin{array}{cccc}{z(0)} & {z\left(\tau_{2}\right)} & {\dots} & {z\left(H 
\tau_{2}\right)} \\ {z^{\prime}(0)} & {z^{\prime}\left(\tau_{2}\right)} & {\dots} 
& {z^{\prime}\left(H \tau_{2}\right)} \\ {z^{(n)}(0)} & {z^{(n)}\left(\tau_{2}\right)} 
& {\dots} & {z^{(n)}\left(H \tau_{2}\right)}\end{array}\right]$$

Such a function can be tested with the following example.

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib ipympl

In [ ]:
def  lqrtrajgeneration(P,T,X0,rho,tau1,N12):

    # initializations 
    N  = np.max(T.shape)
    T  = T-T[0] # without loss of generality we set T(1) = 0
    h  = np.int(np.floor(T[-1]/tau1))
    aux, n  = X0.shape
    rk = np.zeros((3,h+1))
    Ac = np.zeros((n,n))
    Ac[0:n-1,1:n] = np.eye(n-1)
    Bc = np.zeros((n,1))
    Bc[-1] = 1
    Cc = np.zeros((1,n))
    Dc = 0
    A, B = signal.cont2discrete((Ac, Bc, Cc, Dc), tau1)[:2]
    M = np.zeros((1,n))
    M[0][0] = 1
    R = rho
    tau2 = tau1/N12

    # obtain rk 
    i = 0
    for k in range(h+1):
        tk = k*tau1
        if tk > T[i+1]:
            if i+1 == N-1:
                break
            else:
                i +=1
            
        rk[:,k] = P[:,i] + (tk-T[i])/(T[i+1]-T[i])*(P[:,i+1]-P[:,i])

    # obtain optimal control inputs - since the initial problem in the 3D space
    # decouples this amount to solving three reference tracking problems
    rkx = rk[0,:]
    rky = rk[1,:]
    rkz = rk[2,:]
    x0x = X0[0,:]
    x0y = X0[1,:]
    x0z = X0[2,:]

    betah = 1
    ux = lqrreftrack(A,B,M,R,rkx,x0x,betah)
    uy = lqrreftrack(A,B,M,R,rky,x0y,betah)
    uz = lqrreftrack(A,B,M,R,rkz,x0z,betah)

    # obtain derivatives
    Ad, Bd = signal.cont2discrete((Ac, Bc, np.zeros((1,A.shape[0])),0), tau2)[:2]
    xix_ = np.zeros((n,h*N12+1))
    xiy_ = np.zeros((n,h*N12+1)) 
    xiz_ = np.zeros((n,h*N12+1))
    xix_[:,0] = x0x
    xiy_[:,0] = x0y
    xiz_[:,0] = x0z
    ux_ = np.kron(ux,np.ones((1,N12)))
    uy_ = np.kron(uy,np.ones((1,N12))) 
    uz_ = np.kron(uz,np.ones((1,N12)))
    for kappa in range(h*N12):
        xix_[:,kappa+1] = (Ad@xix_[:,kappa].reshape(3,1) + Bd*ux_[0][kappa]).reshape(3)
        xiy_[:,kappa+1] = (Ad@xiy_[:,kappa].reshape(3,1) + Bd*uy_[0][kappa]).reshape(3)
        xiz_[:,kappa+1] = (Ad@xiz_[:,kappa].reshape(3,1) + Bd*uz_[0][kappa]).reshape(3)

    xix = np.vstack([xix_, np.append(ux_,ux_[0,-1])])
    xiy = np.vstack([xiy_, np.append(uy_,uy_[0,-1])])
    xiz = np.vstack([xiz_, np.append(uz_,uz_[0,-1])])
    
    return xix, xiy, xiz

Note that for the last step consisting of providing the output at a faster rate, i.e. a smaller sampling period $\tau_2$, we have discretized the system above at this rate obtaining

$$ \underline{x}_{\ell+1} = \underline{A}\,\underline{x}_\ell + \underline{B}\,\underline{u}_\ell,$$

with $\underline{A} = e^{A_c\tau_2}$, $\underline{B}=\int_0^{\tau_2}e^{A_cs}B_cds$, $\underline{x}_\ell:=x(\ell\tau_2)$, and where the control input is

$$\underline{u}_\ell = u_{\text{fl}[\frac{\ell}{N12}]}$$

for the optimal solution $u_k$ to the optimization problem at sampling period $\tau_1$, where $\text{fl}[a]$ denotes the floor of $a$. 

The following function solves the reference tracking problem, see live script "Reference tracking LQR" for an explanation.

In [ ]:
def lqrreftrack(A,B,M,R,r,x0,betah):
    

    # initializations
    h = max(r.shape)-1
    n = A.shape[0]
    m = B.shape[1]
    p = M.shape[1]
    P = np.zeros((n,n,h+1))
    N = np.zeros((1,n,h+1))
    K = np.zeros((m,n,h))
    L = np.zeros((m,1,h))
    u = np.zeros((m,h))
    x = np.zeros((n,h+1))

    # compute the gains 
    P[:,:,h] = betah*(M.T*M)
    N[:,:,h] = -betah*2*r[h].T*M
    for k in range(h-1,-1,-1):
        P[:,:,k] = M.T@M + A.T@P[:,:,k+1]@A-A.T@P[:,:,k+1]@B@np.linalg.inv(R+B.T@P[:,:,k+1]@B)@B.T@P[:,:,k+1]@A
        K[:,:,k] = -np.linalg.inv(R + B.T @ P[:,:,k+1]@B)@B.T@P[:,:,k+1]@A
        N[:,:,k] = -2*r[k].T*M + N[:,:,k+1]@(A+B@K[:,:,k])
        L[:,:,k] = -np.linalg.inv(R+B.T@P[:,:,k+1]@B)@B.T*1/2@N[:,:,k+1].T

    # compute optimal trajectory (uk)
    x[:,0]   = x0
    for k in range(h):
        u[:,k] = K[:,:,k]@x[:,k] + L[:,:,k][0]
        x[:,k+1] = A@x[:,k] + B@u[:,k]

    return u

In [ ]:
# generate a star like trajectory
thetavec = np.arange(0,5) / 5 * np.pi * 2
thetavec2 = thetavec+1/10 * (2*np.pi)
p = np.zeros((11,2))
for i in range(5):
    p[i*2,0] = 2*np.sin(thetavec[i])
    p[i*2,1] = 2*np.cos(thetavec[i])
    p[i*2+1,0] = np.sin(thetavec2[i])
    p[i*2+1,1] = np.cos(thetavec2[i])

p[10,0] = p[0,0]
p[10,1] = p[0,1]
P = np.zeros((3,11))
P[0:2,:] = p.T
Tend  = 20
aux,N = P.shape
n = 3
T = np.arange(0,N) / (N-1) * Tend
rho = 0.01
tau1 = 0.1
X0 = np.zeros((3,3))
X0[:,0] = P[:,0]
X0[:,1] = (P[:,1]-P[:,0])/T[1]
N12 = 10
# call the function and plot both the initial and the smothened trajectory
xix,xiy,xiz  = lqrtrajgeneration(P,T,X0,rho,tau1,N12)

f = plt.figure()
# swap x/y axes
ax = plt.gca()
ax.plot(P[1,:],P[0,:])
ax.plot(xiy[0,:],xix[0,:])
ax.axis('equal')
plt.xlabel('y')
plt.ylabel('x')
plt.xlim((2, -2))
ax.grid(True)